In [1]:
from my_libs import *

/home/ken/.local/lib/python2.7/site-packages/pyfolio/pos.py:28: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  ' to position notionals.'


In [2]:
first = momentum_best_day("wina",robingateway(),method = 'self_minute').iloc[0]

All price data of Close is actually Adj Close
Connection Successful
('Finished', 'wina')


## Screen

In [4]:
momentum_backtest("wina",10,15,robingateway(),method = 'self_minute')

All price data of Close is actually Adj Close
Connection Successful
('Finished', 'wina')
2020-05-13T15:00:00, BUY CREATE, 132.00, position value: 2000.00
2020-05-13T15:00:00, ORDER ACCEPTED/SUBMITTED
2020-05-13T15:00:00, ORDER ACCEPTED/SUBMITTED
2020-05-13T15:30:00, BUY EXECUTED, Price: 126.38, Cost: 1895.64, Comm 1.90
2020-05-14T15:30:00, SELL CREATE, 125.41, position value: 1983.61
2020-05-14T15:30:00, ORDER ACCEPTED/SUBMITTED
2020-05-14T15:30:00, ORDER ACCEPTED/SUBMITTED
2020-05-14T16:00:00, SELL EXECUTED, Price: 125.41, Gain: 1895.64, Comm 1.88
2020-05-14T16:00:00, BUY CREATE, 125.20, position value: 1981.73
2020-05-14T16:00:00, ORDER ACCEPTED/SUBMITTED
2020-05-14T16:00:00, ORDER ACCEPTED/SUBMITTED
2020-05-15T09:30:00, BUY EXECUTED, Price: 125.81, Cost: 1887.15, Comm 1.89
2020-05-18T09:30:00, SELL CREATE, 127.51, position value: 2005.35
2020-05-18T09:30:00, ORDER ACCEPTED/SUBMITTED
2020-05-18T09:30:00, ORDER ACCEPTED/SUBMITTED
2020-05-18T10:00:00, SELL EXECUTED, Price: 127.51, Gain

(2667.3902510999983,
 14.479548718632866,
 datetime.datetime(2020, 6, 30, 12, 0),
 168.8,
 0.063986073657564,
 -0.03730295984758388)

In [51]:
price=get_price_data(["TCI"], method="self_minute",robinhood=robingateway(),back_day=200 )
price = price.set_index("TimeStamp")
price["MV32"] = price.Close.rolling(32).mean()
price["Return"] = log(price.Close/price.Close.shift(65))
price["Lookback"] = price.MV32.shift(65)

All price data of Close is actually Adj Close
Connection Successful
('Finished', 'TCI')


In [5]:
robinhood = robingateway()
log_pos = get_open_opsition()
temp = robinhood.get_my_positions()
pos_frame = pd.DataFrame([temp[0],temp[1]]).transpose()
pos_frame.columns = ["Ticker","Shares"]
#     act_pos = robinhood.get_my_positions()[0]
note = ""
for i in log_pos:
    if i not in pos_frame.Ticker.to_list():
#             note += ("fix "+ i +"\n")
        fix_unsettled_trade_update(i)
        send_email("Fixed unsettled trade to size 0 " + str(i))
    elif get_trade_log(i)["size"].iloc[0] != pos_frame.loc[pos_frame.Ticker == i,"Shares"].iloc[0]:
        fix_unsettled_trade_update(i, size=pos_frame.loc[pos_frame.Ticker == i,"Shares"].iloc[0], partial=True)
        send_email("Fixed unsettled trade to match size " + str(i))

Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection Successful
Connection

## Live Trading Check

In [8]:
mongod = mongo("all_symbol")

# all_ticker = pd.DataFrame(mongod.db["cantrade"].find()).Ticker.tolist()
result = pd.DataFrame()
all_ticker = mongod.conn.get_data("select * from all_symbol.cantrade").Ticker.tolist()

for i in all_ticker:
    try:
        first = momentum_best_day(i,robinhood=robingateway(),method = "self_minute").iloc[0]
        end_value,sharpratio = momentum_backtest(i,first["Lookback"],first["Holding"],robinhood=robingateway(),method = "self_minute")
        temp = pd.DataFrame([(i,end_value,sharpratio, first["Lookback"],first["Holding"])],columns= ["Ticker","End_Value", "Sharp_Ratio","Lookback","Holding"])
        temp["Refresh_Date"] = datetime.today().date()
        result = result.append(temp)
        mongod.frame_to_mongo(collection_str="momentum_sharp",data=temp,drop_mode="append")
    except Exception as e:
        print (e)

Connection Successful
All price data of Close is actually Adj Close
Connection Successful
('Finished', 'AAPL')
All price data of Close is actually Adj Close
Connection Successful
('Finished', 'AAPL')
2020-05-07T14:00:00, BUY CREATE, 303.99, position value: 2000.00
2020-05-07T14:00:00, ORDER ACCEPTED/SUBMITTED
2020-05-07T14:00:00, ORDER ACCEPTED/SUBMITTED
2020-05-07T14:00:00, ORDER ACCEPTED/SUBMITTED
2020-05-07T14:30:00, BUY EXECUTED, Price: 303.36, Cost: 1820.14, Comm 1.82
2020-05-18T10:00:00, SELL CREATE, 312.09, position value: 2050.58
2020-05-18T10:00:00, ORDER ACCEPTED/SUBMITTED
2020-05-18T10:00:00, ORDER ACCEPTED/SUBMITTED
2020-05-18T10:00:00, ORDER ACCEPTED/SUBMITTED
2020-05-18T10:30:00, SELL EXECUTED, Price: 312.09, Gain: 1820.14, Comm 1.87
2020-05-18T10:30:00, BUY CREATE, 312.67, position value: 2048.71
2020-05-18T10:30:00, ORDER ACCEPTED/SUBMITTED
2020-05-18T10:30:00, ORDER ACCEPTED/SUBMITTED
2020-05-18T10:30:00, ORDER ACCEPTED/SUBMITTED
2020-05-18T11:00:00, BUY EXECUTED, Pric

KeyboardInterrupt: 

In [2]:
mongod = mongo("all_symbol")

# candidates = pd.DataFrame(mongod.db["momentum"].find()).sort_values("End_Value", ascending=False)    

candidates = mongod.conn.get_data('''

SELECT * FROM all_symbol.`Dist_momentum_sharp_TOS` ORDER BY `Sharp_Ratio` DESC

''')    

strategy_name = "momentum"

Connection Successful


In [15]:
for i in candidates.Ticker.iloc[:20]:
    try:
        price = get_price_data([i], method="realtimeday",robinhood=robingateway(),back_day=200 )
        price = price.set_index("TimeStamp")
        lookback = int(candidates.loc[candidates.Ticker == i,"Lookback"].iloc[0])
        holddays = int(candidates.loc[candidates.Ticker == i,"Holding"].iloc[0])
#     price["Return"] = np.log(price.Close/price.Close.shift(lookback))

        fprice = np.log(fwd_price(i,mat=14,val_num_steps=30)/price.Close.iloc[-lookback])
    except Exception as e:
        send_email("Price in momentum check error %s, Error: %s. Process continues"%(i,str(e)))
        continue

All price data of Close is actually Adj Close
('Finished', u'PHT')
Ascending
All price data of Close is actually Adj Close
('Finished', u'PHT')
All price data of Close is actually Adj Close
('Finished', u'BITA')
Ascending
2020-07-17
Ascending
Going ATP
current option:
 Symbol:BITA strike:17.5, expiration:2020-07-17, type:call
current option:
 Symbol:BITA strike:17.5, expiration:2020-07-17, type:call


my_libs/option.py:217: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  get.set_value("break_even_pert",(float(get.break_even_price) - float(self.strike))/float(self.strike))


All price data of Close is actually Adj Close
('Finished', u'WINA')
Ascending
All price data of Close is actually Adj Close
('Finished', u'WINA')
All price data of Close is actually Adj Close
('Finished', u'DRD')
Ascending
All price data of Close is actually Adj Close
('Finished', u'DRD')
All price data of Close is actually Adj Close
('Finished', u'SPWH')
Ascending
2020-07-17
Ascending
Going ATP
current option:
 Symbol:SPWH strike:15.0, expiration:2020-07-17, type:call
current option:
 Symbol:SPWH strike:15.0, expiration:2020-07-17, type:call
All price data of Close is actually Adj Close
('Finished', u'VAPO')
Ascending
2020-07-17
Ascending
Going ATP
current option:
 Symbol:VAPO strike:45.0, expiration:2020-07-17, type:call
current option:
 Symbol:VAPO strike:45.0, expiration:2020-07-17, type:call
All price data of Close is actually Adj Close
('Finished', u'SLG')
Ascending
2020-07-17
Ascending
Going ATP
current option:
 Symbol:SLG strike:50.0, expiration:2020-07-17, type:call
current op

## Test

In [3]:
mongod = mongo("all_symbol")

# candidates = pd.DataFrame(mongod.db["momentum"].find()).sort_values("End_Value", ascending=False)    

cantrade = mongod.conn.get_data('''

SELECT ticker FROM all_symbol.cantrade

''')    


Connection Successful


In [19]:
result = pd.DataFrame()
for i in cantrade.ticker:
    ddcf = dcf(i,"today")
    if len(ddcf)==0:
        continue
    temp = ddcf.set_index("date")

    temp["Growth%"] = np.log(temp["dcf"]/temp["Stock Price"])

    temp = temp.round(3)
    
    result = result.append(temp)

Cannot convert datetime to date
Cannot convert datetime to date
Cannot convert datetime to date
Cannot convert datetime to date
Cannot convert datetime to date
Cannot convert datetime to date


KeyboardInterrupt: 

In [13]:
financial_ratios(i).iloc[0]

Cannot convert datetime to date


assetTurnover                                      0.0201
capitalExpenditureCoverageRatio                       NaN
cashConversionCycle                                   NaN
cashFlowCoverageRatios                             0.2524
cashFlowToDebtRatio                                0.2524
cashPerShare                                      67.4892
cashRatio                                         40.9947
companyEquityMultiplier                            9.1918
currentRatio                                     175.0048
date                                  2019-12-31 00:00:00
daysOfInventoryOutstanding                            NaN
daysOfPayablesOutstanding                             NaN
daysOfSalesOutstanding                           417.6025
debtEquityRatio                                    8.1918
debtRatio                                          0.8912
dividendPaidAndCapexCoverageRatio                     NaN
dividendPayoutRatio                                0.2241
dividendYield 

In [18]:
dcf(i,"today")

Cannot convert datetime to date


""


In [ ]:
plot_ticker("AMGN",method="self_minute", robinhood = robingateway(),past_days=200)

In [119]:
mean_reversion(1,1,usr_input=price.Close)

If you use outside data feed please enter random inputs for the first two arguments
***********************************************
**********1*******************
Augmented Dickey Fuller test statistic = -1.2996900537418
Augmented Dickey Fuller p-value = 0.6293141025389983
Augmented Dickey Fuller 1%, 5% and 10% test statistics = {'5%': -2.88319822181578, '1%': -3.479742586699182, '10%': -2.578319684499314}
0.29745205944529
Halflife =  47.0


['Augmented Dickey Fuller: Not significant', 'Hurst Exponent:mean reverting', 'Halflife: 47.0 1']
***********************************************


[['Augmented Dickey Fuller: Not significant',
  'Hurst Exponent:mean reverting',
  'Halflife: 47.0 1'],
 47.0]

In [7]:
holding = 10
lookback = 5
shorter = holding if holding <= lookback else lookback
price = get_price_data([ticker], method = "day", robinhood=robingateway(),back_day=200)
price = price.set_index("TimeStamp")

All price data of Close is actually Adj Close
Connection Successful
('Finished', 'AAPL')


In [34]:
ret_pre= log(price.Close/price.Close.shift(lookback))

ret_pre = ret_pre.fillna(0)

ret_post = log(price.Close.shift(-holding)/price.Close)

ret_post = ret_post.fillna(0)

# ret_pre= ret_pre.iloc[:-shorter]

# ret_post= ret_post.iloc[shorter:]


In [35]:
stats.pearsonr(ret_pre,ret_post)

(0.16665874090950547, 0.05336855345564748)

statsmodels.stats.stattools.durbin_watson(price.iloc[:-100].Return.dropna())